In [28]:
import os
from time import sleep


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager


from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup

##############################################################  ############
##################### variable related selenium ##########################
##########################################################################
options = webdriver.ChromeOptions()
# options.add_argument('headless')
# options.add_argument('lang=ko_KR')
chromedriver_path = "/Users/seongrok.kim/Github/CrawlerList/chromedriver-mac-x64/chromedriver"
# chromedriver_path = ChromeDriverManager().install()
driver = webdriver.Chrome(chromedriver_path, chrome_options=options)
page_num = 1
url = f'https://onlineonly.christies.com/s/inside-job-christies-new-york-staff-art-show/lots/3728'

driver.get(url)

# for page_num in range(1,4286):
#     url = f'https://filmarks.com/movies/58836?page={page_num}'

#     driver.get(url)

<ipython-input-28-a9777f781e5d>:27: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver_path, chrome_options=options)
<ipython-input-28-a9777f781e5d>:27: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chromedriver_path, chrome_options=options)


## selenium 스크래핑 방법
### 둘의 차이 알기
- 단일건 가져오기
  - driver.find_element()
- 리스트로 가져오기
  - driver.find_elements()

In [29]:
link_elements = driver.find_elements(
        By.XPATH, f"//li[@class='col-12 col-md-4 col-lg-3 col-xl-20pct chr-browse-lot-tile-wrapper']"
    )

len(link_elements)

84

In [30]:
result_list = []

for link_element in link_elements:
    title_elements = link_element.find_elements(By.CLASS_NAME, 'chr-lot-tile__titles')
    # secound_title_elements = link_element.find_elements(By.CLASS_NAME, 'chr-lot-tile__secondary-title ellipsis--one-line')
    price_elements = link_element.find_elements(By.CLASS_NAME, 'chr-lot-tile__dynamic-price')
    if title_elements and price_elements:
        title = title_elements[0].get_attribute('innerText')
        price = price_elements[0].get_attribute('innerText')
        
        result_dict = {
            "title": title,
            "price": price
        }
        result_list.append(result_dict)
        print('tilte', title)
        print('price', price)

tilte RICARDO JAÉN

Las Tres Gracias
price Price realised
USD 1,260
tilte RICARDO JAÉN

Living Room (Still Life)
price 
tilte HANS WOLF-GUZMAN

Was Gehtz Berlin?
price 
tilte HANS WOLF-GUZMAN

Graphic Alliteration
price 
tilte GRACE DALE

Baring Witness
price Price realised
USD 630
tilte ANDREA LUPER

Eternal Return
price Price realised
USD 630
tilte ANDREA LUPER

Hang
price Price realised
USD 1,638
tilte BLUSTERONE

Make 'Em Clap to This
price 
tilte JOE EGAN

Sian Ka'an Spirit
price 
tilte MICHAEL WALSH

Adjacent
price 
tilte MICHAEL WALSH

Hex Omega
price 
tilte C.C. CASTRO

Canvas of a Military Modeler
price Price realised
USD 252
tilte JOE BARNES

Faded Plaid
price Price realised
USD 126
tilte JOE BARNES

Arizona Skies
price Price realised
USD 126
tilte NEIL CONSTANTINE

Redrum
price 
tilte NEIL CONSTANTINE

Lost Yet Calm
price Price realised
USD 4,410
tilte ERIC MAVKO

2XANNA
price Price realised
USD 504
tilte ABIGAIL LIEBHART

Untitled
price 
tilte ERIC MAVKO

Cat#1
price Price 

In [31]:
result_list

[{'title': 'RICARDO JAÉN\n\nLas Tres Gracias',
  'price': 'Price realised\nUSD 1,260'},
 {'title': 'RICARDO JAÉN\n\nLiving Room (Still Life)', 'price': ''},
 {'title': 'HANS WOLF-GUZMAN\n\nWas Gehtz Berlin?', 'price': ''},
 {'title': 'HANS WOLF-GUZMAN\n\nGraphic Alliteration', 'price': ''},
 {'title': 'GRACE DALE\n\nBaring Witness', 'price': 'Price realised\nUSD 630'},
 {'title': 'ANDREA LUPER\n\nEternal Return',
  'price': 'Price realised\nUSD 630'},
 {'title': 'ANDREA LUPER\n\nHang', 'price': 'Price realised\nUSD 1,638'},
 {'title': "BLUSTERONE\n\nMake 'Em Clap to This", 'price': ''},
 {'title': "JOE EGAN\n\nSian Ka'an Spirit", 'price': ''},
 {'title': 'MICHAEL WALSH\n\nAdjacent', 'price': ''},
 {'title': 'MICHAEL WALSH\n\nHex Omega', 'price': ''},
 {'title': 'C.C. CASTRO\n\nCanvas of a Military Modeler',
  'price': 'Price realised\nUSD 252'},
 {'title': 'JOE BARNES\n\nFaded Plaid', 'price': 'Price realised\nUSD 126'},
 {'title': 'JOE BARNES\n\nArizona Skies', 'price': 'Price realise

In [32]:
df = pd.DataFrame(result_list)


In [35]:
def split_titles_to_secound(title):
    if title:
        return title.split('\n\n')[-1]
    else:
        return title
    
def split_titles(title):
    if title:
        return title.split('\n\n')[0]
    else:
        return title

In [36]:
# title 컬럼 분리
df['second_title'] = df['title'].apply(split_titles_to_secound)
df['title'] = df['title'].apply(split_titles)


# price 컬럼에서 'Price realised\n' 문자열 제거
df['price'] = df['price'].str.replace('Price realised\n', '', regex=False)


In [38]:
df.to_csv('onlineonly.csv', index=False, encoding='utf-8-sig')